In [1]:
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
from functools import reduce
import re
import numpy as np

In [2]:
DemandTracker=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\DemandTracker.xlsx')
OnboardingTracker=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\OnboardingTracker.xlsx')
HCL=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\HCL.xlsx')
INFY=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\Infy.xlsx')

In [3]:
onshorePublished = DemandTracker[((DemandTracker['Location']=='ANZ - Melbourne'))]
OnRolPub=onshorePublished.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OnshoreRolePublished')

In [4]:
OnRolPub

,Tribe,Role,OnshoreRolePublished
0,APRA,Analyst,1
1,APRA,Engineer,4
2,APRA,Operations Expert,1
3,APRA,Solution Designer,2
4,APRA,Tech Area Architect,1
5,BUSINESS LENDING,Solution Designer,2
6,BUSINESS OWNERS,Solution Designer,2
7,BX - CC,Release Lead,1
8,BX - CSP,Release Lead,1
9,BX - CSP,SDET - Automation Tester,5


In [5]:
onshorePublishedClosed = DemandTracker[((DemandTracker['Location']=='ANZ - Melbourne') & (DemandTracker['Demand Status']=='Closed'))]
OnRolPubClosed=onshorePublishedClosed.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OnshoreRolePublishedClosed')

In [6]:
OnRolPubClosed

,Tribe,Role,OnshoreRolePublishedClosed
0,APRA,Analyst,1
1,APRA,Engineer,3
2,APRA,Operations Expert,1
3,APRA,Solution Designer,2
4,BUSINESS LENDING,Solution Designer,1
5,BUSINESS OWNERS,Solution Designer,2
6,BX - CC,Release Lead,1
7,BX - CSP,Release Lead,1
8,BX - CSP,SDET - Automation Tester,1
9,BX - CSP,Test Lead,1


In [7]:
DXCOnboarding = OnboardingTracker
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'

In [8]:
OnshoreResign = DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Resigned') & (DXCOnboarding['Location']=='ANZ - Melbourne'))]
OnshoreResign=OnshoreResign.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OnshoreResigned')

In [9]:
OnshoreResign

,Tribe,Role,OnshoreResigned
0,BX - CSP,Agile Coach,1
1,BX - CSP,Analyst,1
2,BX - CSP,Engineer,3
3,DCX,Engineer,2
4,DSE,Chapter Lead,1
5,SOE,Agile Coach,1


In [10]:
OnshoreNoshow = DXCOnboarding[((DXCOnboarding['Onboarding Status']=='No Show') & (DXCOnboarding['Location']=='ANZ - Melbourne'))]
OnshoreNoshow=OnshoreNoshow.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OnshoreDropOff')

In [11]:
OnshoreNoshow

,Tribe,Role,OnshoreDropOff
0,APRA,Solution Designer,1
1,BUSINESS LENDING,Solution Designer,1
2,DCX,Engineer,1
3,SOE,Agile Coach,1


In [12]:
OnshoreHired = DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Onboarding Completed') & (DXCOnboarding['Location']=='ANZ - Melbourne'))]
OnshoreHired=OnshoreHired.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OnshoreHired')

In [13]:
df=[OnRolPub,OnRolPubClosed,OnshoreResign,OnshoreNoshow,OnshoreHired]

In [14]:
onshore=reduce(lambda left,right: pd.merge(left,right,on=['Tribe','Role'],how='outer'),df)

In [15]:
onshore[['OnshoreRolePublished','OnshoreRolePublishedClosed','OnshoreResigned','OnshoreDropOff','OnshoreHired']]=onshore[['OnshoreRolePublished','OnshoreRolePublishedClosed','OnshoreResigned','OnshoreDropOff','OnshoreHired']].fillna(0)

In [16]:
onshore[['OnshoreRolePublished','OnshoreRolePublishedClosed','OnshoreResigned','OnshoreDropOff','OnshoreHired']]=onshore[['OnshoreRolePublished','OnshoreRolePublishedClosed','OnshoreResigned','OnshoreDropOff','OnshoreHired']].astype(int)

In [17]:
onshore

,Tribe,Role,OnshoreRolePublished,OnshoreRolePublishedClosed,OnshoreResigned,OnshoreDropOff,OnshoreHired
0,APRA,Analyst,1,1,0,0,1
1,APRA,Engineer,4,3,0,0,2
2,APRA,Operations Expert,1,1,0,0,1
3,APRA,Solution Designer,2,2,0,1,0
4,APRA,Tech Area Architect,1,0,0,0,0
5,BUSINESS LENDING,Solution Designer,2,1,0,1,0
6,BUSINESS OWNERS,Solution Designer,2,2,0,0,0
7,BX - CC,Release Lead,1,1,0,0,2
8,BX - CSP,Release Lead,1,1,0,0,6
9,BX - CSP,SDET - Automation Tester,5,1,0,0,0


In [18]:
onshore.to_csv(r'C:\NotBackedUp\SOE\pyscript\Tribe\OnshoreRoles.csv',sep=',',index=None)

In [19]:
OnshoreHired = DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Onboarding Completed') & (DXCOnboarding['Location']=='ANZ - Melbourne'))]
OnshoreHired=OnshoreHired.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OnshoreHired')

In [21]:
OffshoreHired = DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Onboarding Completed') & (DXCOnboarding['Location'].isin(['ANZOT - Bangalore','HCL ODC - Bangalore','Infosys ODC - Pune'])))]
OffshoreHired=OffshoreHired.groupby(['Tribe','Role'])['Demand ID'].count().reset_index(name='OffshoreHired')

In [22]:
d1=[OnshoreHired,OffshoreHired]

In [23]:
hired=reduce(lambda left,right: pd.merge(left,right,on=['Tribe','Role'],how='outer'),d1)

In [25]:
hired[['OnshoreHired','OffshoreHired']] = hired[['OnshoreHired','OffshoreHired']].fillna(0)

In [26]:
hired[['OnshoreHired','OffshoreHired']] = hired[['OnshoreHired','OffshoreHired']].astype(int)

In [28]:
hired.to_csv(r'C:\NotBackedUp\SOE\pyscript\Tribe\RolesHired.csv',sep=',',index=None)

In [29]:
hired

,Tribe,Role,OnshoreHired,OffshoreHired
0,APRA,Analyst,1,2
1,APRA,Engineer,2,3
2,APRA,Operations Expert,1,0
3,BUSINESS LENDING,Engineer,4,0
4,BUSINESS LENDING,System Analyst,2,0
5,BX - CC,Release Lead,2,1
6,BX - CSP,Analyst,4,5
7,BX - CSP,Engineer,23,14
8,BX - CSP,Operations Expert,2,0
9,BX - CSP,Release Lead,6,9


In [3]:
HCLHires = HCL
#HCLHires.loc[HCLHires['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
#HCLHires.loc[HCLHires['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
#HCLHires.loc[HCLHires['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
#HCLHires.loc[HCLHires['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'

In [4]:
INFYHires = INFY
#INFYHires.loc[INFYHires['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
#INFYHires.loc[INFYHires['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
#INFYHires.loc[INFYHires['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
#INFYHires.loc[INFYHires['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'

In [5]:
HCLHires = HCLHires[((HCLHires['Application Status']=='Hired'))]
HCLHires=HCLHires.groupby(['Tribe','Role'])['ID'].count().reset_index(name='HCLHired')

In [6]:
INFYHires = INFYHires[((INFYHires['Application Status']=='Hired'))]
INFYHires=INFYHires.groupby(['Tribe','Role'])['ID'].count().reset_index(name='InfyHired')

In [7]:
hire = [HCLHires,INFYHires]

In [8]:
h1=reduce(lambda left,right: pd.merge(left,right,on=['Tribe','Role'],how='outer'),hire)

In [9]:
h1[['HCLHired','InfyHired']] = h1[['HCLHired','InfyHired']].fillna(0)
h1[['HCLHired','InfyHired']] = h1[['HCLHired','InfyHired']].astype(int)

In [10]:
h1['Total'] = h1['HCLHired']+h1['InfyHired']

In [11]:
h1.to_csv(r'C:\NotBackedUp\SOE\pyscript\Tribe\H1.csv',sep=',',index=None)